In [2]:
import numpy as np
import nibabel as nib
import pandas as pd
import os,sys
import math
import os.path
import cv2
import pydicom
from PIL import Image
import cv2 as cv
import random
import pickle
import time
import pydicom
from scipy import ndimage as ndi
#from torch import absolute_eigenvaluesh
from scipy import absolute
from itertools import combinations_with_replacement
from PIL import ImageEnhance
import copy

In [3]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import datasets, layers, optimizers, Sequential, metrics
from tensorflow.python.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import Conv3D
from tensorflow.keras.layers import MaxPooling2D,MaxPooling3D,Input,GlobalAvgPool2D,GlobalAvgPool3D,Activation,BatchNormalization
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.python.keras.utils import np_utils
from tensorflow.keras.callbacks import LearningRateScheduler
import tensorflow.keras.backend as K
from tensorflow.keras.layers import GaussianNoise
from tensorflow.keras.models import load_model
from skimage.transform import resize
from skimage import io, transform
from numba import jit
import copy

In [4]:
d3_plat_1 = [[0,0,0],[-1,-1,0],[0,-1,0],[1,-1,0],[1,0,0],[1,1,0],[0,1,0],[-1,1,0],[-1,0,0],
             [0,0,1],[-1,-1,1],[0,-1,1],[1,-1,1],[1,0,1],[1,1,1],[0,1,1],[-1,1,1],[-1,0,1],
             [0,0,-1],[-1,-1,-1],[0,-1,-1],[1,-1,-1],[1,0,-1],[1,1,-1],[0,1,-1],[-1,1,-1],[-1,0,-1]]
d2_plat_1 = [[0,0,0],[-1,-1,0],[0,-1,0],[1,-1,0],[1,0,0],[1,1,0],[0,1,0],[-1,1,0],[-1,0,0]]
d3_plat_1_5 = [[0,0,0],[-5,-5,0],[0,-5,0],[5,-5,0],[5,0,0],[5,5,0],[0,5,0],[-5,5,0],[-5,0,0],
               [0,0,5],[-5,-5,5],[0,-5,5],[5,-5,5],[5,0,5],[5,5,5],[0,5,5],[-5,5,5],[-5,0,5],
               [0,0,-5],[-5,-5,-5],[0,-5,-5],[5,-5,-5],[5,0,-5],[5,5,-5],[0,5,-5],[-5,5,-5],[-5,0,-5]]
d3_plat_1_no0 = [[-1,-1,0],[0,-1,0],[1,-1,0],[1,0,0],[1,1,0],[0,1,0],[-1,1,0],[-1,0,0],
             [0,0,1],[-1,-1,1],[0,-1,1],[1,-1,1],[1,0,1],[1,1,1],[0,1,1],[-1,1,1],[-1,0,1],
             [0,0,-1],[-1,-1,-1],[0,-1,-1],[1,-1,-1],[1,0,-1],[1,1,-1],[0,1,-1],[-1,1,-1],[-1,0,-1]]
d3_plat_0 = [[0,0,0]]
d2_plat_1_4 = [[0,-1],[1,0],[0,1],[-1,0]]
d2_plat_1_8 = [[-1,-1],[0,-1],[1,-1],[1,0],[1,1],[0,1],[-1,1],[-1,0]]
d3_plat_8 = [[0,0,0],[1,0,0],[1,1,0],[0,1,0],
             [0,0,1],[1,0,1],[1,1,1],[0,1,1]]
d3_plat_8_no0 = [[1,0,0],[1,1,0],[0,1,0],
             [0,0,1],[1,0,1],[1,1,1],[0,1,1]]
six_direction = [[-1,0,0],[1,0,0],[0,-1,0],[0,1,0],[0,0,-1],[0,0,1]]
small_patch = 15
big_patch = 64
patch0 = small_patch
patch1 = small_patch
patch2 = small_patch
patch3 = big_patch
patch4 = big_patch
patch5 = big_patch

In [7]:
windows = 0

In [6]:
def read_data(path):#定义一个“读取图像”函数，用来读取图像
    image_data = nib.load(path)
    return image_data

In [27]:
file_path1 = r'/home/zxli/ssjm/P'#r'E:\article4\样例文件\P'
file_path0 = r'/home/zxli/ssjm/P'
file_number = '10'
left_x = 205
right_x = 345
left_low = 70
left_high = 110
right_low = 80
right_high = 120
gradient_threshold = 13
xb1,xb2 = 60,50
yb1,yb2 = 80,60
zb1,zb2 = 16,16
coregister = [44, 194, 102, 224, 216, 248]
t1_threshold = 1000
t1_type = 0
xt1,xt2,yt1,yt2,zt1,zt2 = 78,184,103,226,203,243
thresould_low,thresould_high = max(left_low,right_low),min(left_high,right_high)

In [9]:
def get_image(file_number):
    if windows == 1:
        file_path = file_path1
        path_p02 = file_path+file_number+'\dp'+file_number+'_mag_insert2_n4.nii'#图像路径，日后可以根据需要修改
    else:
        file_path = file_path0
        path_p02 = file_path+file_number+'/dp'+file_number+'_mag_insert2_n4.nii'#图像路径，日后可以根据需要修改
    p02_img = read_data(path_p02)#使用“读取图像”函数

    p02_3d = p02_img.get_fdata()
    p02_3d.shape
    return p02_3d

In [10]:
def get_region(file_path,file_number):
    begin_time = time.time()
    sembles = []
    for m in range(5):
        if windows == 1:
            path = file_path+file_number+'\dp'+file_number+'_mag_insert%s_n4.nii'%m#图像路径，日后可以根据需要修改
            #print(path)
        else:
            path = file_path+file_number+'/dp'+file_number+'_mag_insert%s_n4.nii'%m#图像路径，日后可以根据需要修改
        target = read_data(path)#使用“读取图像”函数

        target_3d = target.get_fdata()
        target_3d.shape

        target_3d_normalization = (target_3d/1000)*1000
        sembles.append(target_3d_normalization)
    print(time.time()-begin_time)
    return sembles

In [11]:
def get_region_pha(file_path,file_number):
    begin_time = time.time()
    sembles = []
    for m in range(5):
        if windows == 1:
            path = file_path+file_number+'\dp'+file_number+'_phase_insert%s_n4.nii'%m#图像路径，日后可以根据需要修改
            #print(path)
        else:
            path = file_path+file_number+'/dp'+file_number+'_phase_insert%s_n4.nii'%m#图像路径，日后可以根据需要修改
        target = read_data(path)#使用“读取图像”函数

        target_3d = target.get_fdata()
        target_3d.shape

        target_3d_normalization = (target_3d/1000)*1000
        sembles.append(target_3d_normalization)
    print(time.time()-begin_time)
    return sembles

In [12]:
path_t1 = file_path1+file_number+'/dp'+file_number+'_t1.nii'
target_t1 = read_data(path_t1)#使用“读取图像”函数

target_3d_t1 = target_t1.get_fdata()
target_3d_t1.shape

FileNotFoundError: No such file or no access: 'E:\article4\样例文件\P18/dp18_t1.nii'

In [13]:
def compute_3d_hessian_matrix(nd_array, sigma=1, scale=True, whiteonblack=True):
    """
    Computes the hessian matrix for an nd_array.
    This can be used to detect vesselness as well as other features.
    In 3D the first derivative will contain three directional gradients at each index:
    [ gx,  gy,  gz ]
    The Hessian matrix at each index will then be equal to the second derivative:
    [ gxx, gxy, gxz]
    [ gyx, gyy, gyz]
    [ gzx, gzy, gzz]
    The Hessian matrix is symmetrical, so gyx == gxy, gzx == gxz, and gyz == gzy.
    :param nd_array: n-dimensional array from which to compute the hessian matrix.
    :param sigma: gaussian smoothing to perform on the array.
    :param scale: if True, the hessian elements will be scaled by sigma squared.
    :return: hessian array of shape (..., ndim, ndim)
    """
    ndim = nd_array.ndim

    # smooth the nd_array
    smoothed = ndi.gaussian_filter(nd_array, sigma=sigma, mode='nearest', truncate=3.0)

    # compute the first order gradients
    gradient_list = np.gradient(smoothed)

    # compute the hessian elements
    hessian_elements = [np.gradient(gradient_list[ax0], axis=ax1)
                        for ax0, ax1 in combinations_with_replacement(range(ndim), 2)]

    if sigma > 0 and scale:
        # scale the elements of the hessian matrix
        if whiteonblack:
            hessian_elements = [(sigma ** 2) * element for element in hessian_elements]
        else:
            hessian_elements = [-1 * (sigma ** 2) * element for element in hessian_elements]

    # create hessian matrix from hessian elements
    hessian_full=[[()] * ndim for x in range(ndim)]
    #hessian_full = [[None] * ndim] * ndim

    for index, (ax0, ax1) in enumerate(combinations_with_replacement(range(ndim), 2)):
        element = hessian_elements[index]
        hessian_full[ax0][ax1] = element
        if ax0 != ax1:
            hessian_full[ax1][ax0] = element

    hessian_rows = list()
    for row in hessian_full:
        #print(row.shape)
        hessian_rows.append(np.stack(row, axis=-1))

    hessian = np.stack(hessian_rows, axis=-2)
    return hessian


def absolute_3d_hessian_eigenvalues(nd_array, sigma=1, scale=True, whiteonblack=True):
    """
    Eigenvalues of the hessian matrix calculated from the input array sorted by absolute value.
    :param nd_array: input array from which to calculate hessian eigenvalues.
    :param sigma: gaussian smoothing parameter.
    :param scale: if True hessian values will be scaled according to sigma squared.
    :return: list of eigenvalues [eigenvalue1, eigenvalue2, ...]
    """
    return np.absolute(compute_3d_hessian_matrix(nd_array, sigma=sigma, scale=scale, whiteonblack=whiteonblack))

In [14]:
def make_region(points,imgs,imgs_pha):
    region_semble = []
    region_pha_semble = []
    big_region_semble = []
    hessian_semble = []
    a,b,c = target_same_insert.shape
    for n in range(len(points)):
        x,y,z = points[n]
        if int(patch3/2)<x<a-int(patch3/2) and int(patch3/2)<y<b-int(patch3/2) and int(patch3/2)<z<c-int(patch3/2):
            regions = []
            for m in range(5):
                region = np.zeros((patch0,patch1,patch2))
                region = imgs[m][int(x-(patch0-1)/2):int(x-(patch0-1)/2)+patch0,int(y-(patch1-1)/2):int(y-(patch1-1)/2)+patch1,int(z-(patch2-1)/2):int(z-(patch2-1)/2)+patch2]
                region = np.array(region)
                #print('make_region process',region.shape)
                region_max = get_max(region)
                region = region/region_max
                regions.append(region)
            region_semble.append(regions)
            regions_pha = []
            for m in range(5):
                region_pha = np.zeros((patch0,patch1,patch2))
                region_pha = imgs_pha[m][int(x-(patch0-1)/2):int(x-(patch0-1)/2)+patch0,int(y-(patch1-1)/2):int(y-(patch1-1)/2)+patch1,int(z-(patch2-1)/2):int(z-(patch2-1)/2)+patch2]
                region_pha = np.array(region_pha)
                region_max_pha = get_max(region_pha)
                region_pha = region_pha/region_max_pha
                regions_pha.append(region_pha)
            region_pha_semble.append(regions_pha)
            big_regions = []
            for m in range(2,3):
                big_region = np.zeros((patch3,patch4,patch5))
                big_region = imgs[m][int(x-(patch3-1)/2):int(x-(patch3-1)/2)+patch3,int(y-(patch4-1)/2):int(y-(patch4-1)/2)+patch4,int(z-(patch5-1)/2):int(z-(patch5-1)/2)+patch5]
                big_region = np.array(big_region)
                big_region_max = get_max(big_region)
                big_region = big_region/big_region_max
                big_regions.append(big_region)
            big_region_semble.append(big_regions)
            
            ifile = imgs[2]
#             region_big = ifile[int(x-(patch3-1)/2):int(x-(patch3-1)/2)+patch3,int(y-(patch4-1)/2):int(y-(patch4-1)/2)+patch4,int(z-(patch4-1)/2):int(z-(patch4-1)/2)+patch4]
#             region_big = np.array(region_big)
#             region_big_max = get_max(region_big)
#             region_big = region_rdf/region_big_max
#             region_big_semble.append(region_big)
            
            step = int((patch0-1)/2)
            cita = 1.5
            xc,yc,zc = x,y,z
            if step<xc<a-step and step<yc<b-step and step<zc<c-step:
                array = ifile[xc-step:xc+step,yc-step:yc+step,zc-step:zc+step]
                result = compute_3d_hessian_matrix(array, sigma=cita, scale=True, whiteonblack=True)
                A = np.mat(result[step][step][step])
                U = A*A.T
                lamda,hU=np.linalg.eig(U)
                sigma=np.sqrt(lamda)
                result = [sigma[0],sigma[1],sigma[2]]
                result.sort()
                result2 = np.zeros((3,3,3))
                for i in range(3):
                    result2[i] = np.array([[result[i],result[i],result[i]],[result[i],result[i],result[i]],[result[i],result[i],result[i]]])
            else:
                result2 = np.zeros((3,3,3))
            hessian_semble.append(result2)
        else:
            pass
    return region_semble,region_pha_semble,big_region_semble,hessian_semble#,region_big_semble

In [15]:
if windows == 1:
    file_path = file_path1
else:
    file_path = file_path0
target_tests = get_region(file_path,file_number)
target_tests_pha = get_region_pha(file_path,file_number)
target_same_insert = get_image(file_number)

4.343395471572876
3.8490355014801025


In [ ]:
target_3d_nodenoise = target_same_insert

In [16]:
def MIP_and_gradient(target_3d_1,x_left,x_right,low=30,high=70,threshold=gradient_threshold,up_down=5,left=-1):
    a,b,c = target_3d_1.shape
    positive_train = []
    positive_must = []
    #     [],[],[],[],[],[],[],[],[],[],
    #     [],[],[],[],[],[],[],[],[],[],
    #     [],[],[],[],[],[],[],[],[],[],
    x_ini = 0
    y_ini = 0
    z_ini = 0
    mip = 10
    #threshold = 12
    #up_down = 5

    for i in range(xb1,a-xb2):
        for j in range(yb1,b-yb2):
            for k in range(zb1,c-zb2):
                ready = False
                #候选点
                if low<target_3d_1[x_ini+i,y_ini+j,z_ini+k]<=high and ((i<=x_left and left<0) or (i>=x_right and left>0)):
                    for n in range(int(up_down)):
                        if target_3d_1[x_ini+i,y_ini+j,z_ini+k]+mip<=target_3d_1[x_ini+i,y_ini+j,z_ini+k-int((up_down-1)/2)+n] and -int((up_down-1)/2)+n != 0:
                            #positive_train.append([x_ini+i,y_ini+j,z_ini+k])
                            ready = True
                            break

                    if ready == True:
                        gradient_threshold = 4
                    else:
                        gradient_threshold = 3
                    #从上到下

                    down1 = 0
                    for m in range(4,-1,-1):
                        dy1 = target_3d_1[x_ini+i,y_ini+j+m,z_ini+k]-target_3d_1[x_ini+i,y_ini+j+m+1,z_ini+k]
                        if dy1 < -threshold:
                            down1 = 1

                        elif dy1 > threshold:
                            down1 = 0


                    #从下到上

                    down2 = 0
                    for m in range(-4,1):
                        dy2 = target_3d_1[x_ini+i,y_ini+j+m,z_ini+k]-target_3d_1[x_ini+i,y_ini+j+m-1,z_ini+k]
                        if dy2 < -threshold:
                            down2 = 1

                        elif dy2 > threshold:
                            down2 = 0


                    #从左到右

                    down3 = 0
                    for m in range(-4,1):
                        dx3 = target_3d_1[x_ini+i+m,y_ini+j,z_ini+k]-target_3d_1[x_ini+i+m-1,y_ini+j,z_ini+k]
                        if dx3 < -threshold:
                            down3 = 1

                        elif dx3 > threshold:
                            down3 = 0


                    #从右到左

                    down4 = 0
                    for m in range(4,-1,-1):
                        dx4 = target_3d_1[x_ini+i,y_ini+j+m,z_ini+k]-target_3d_1[x_ini+i,y_ini+j+m+1,z_ini+k]
                        if dx4 < -threshold:
                            down4 = 1

                        elif dx4 > threshold:
                            down4 = 0

                    #左斜

                    down5 = 0
                    for m in range(-4,1):
                        dxy5 = target_3d_1[x_ini+i+m,y_ini+j-m,z_ini+k]-target_3d_1[x_ini+i+m-1,y_ini+j-m+1,z_ini+k]
                        if dxy5 < -threshold:
                            down5 = 1

                        elif dxy5 > threshold:
                            down5 = 0

                    down6 = 0
                    for m in range(4,-1,-1):
                        dxy6 = target_3d_1[x_ini+i+m,y_ini+j-m,z_ini+k]-target_3d_1[x_ini+i+m+1,y_ini+j-m-1,z_ini+k]
                        if dxy6 < -threshold:
                            down6 = 1

                        elif dxy6 > threshold:
                            down6 = 0


                    #右斜

                    down7 = 0
                    for m in range(-4,1):
                        dxy7 = target_3d_1[x_ini+i+m,y_ini+j+m,z_ini+k]-target_3d_1[x_ini+i+m-1,y_ini+j+m-1,z_ini+k]
                        if dxy7 < -threshold:
                            down7 = 1

                        elif dxy7 > threshold:
                            down7 = 0

                    down8 = 0
                    for m in range(4,-1,-1):
                        dxy8 = target_3d_1[x_ini+i+m,y_ini+j+m,z_ini+k]-target_3d_1[x_ini+i+m+1,y_ini+j+m+1,z_ini+k]
                        if dxy8 < -threshold:
                            down8 = 1

                        elif dxy8 > threshold:
                            down8 = 0

                    down_matrix = [down1,down2,down3,down4,down5,down6,down7,down8]
                    down_matrix.sort()
                    if down_matrix[gradient_threshold]==0:
                        pass
                    else:
                        if down1+down2 == 2:
                            positive_train.append([x_ini+i,y_ini+j,z_ini+k])
                        elif down3+down4 == 2:
                            positive_train.append([x_ini+i,y_ini+j,z_ini+k])
                        elif down5+down6 == 2:
                            positive_train.append([x_ini+i,y_ini+j,z_ini+k])
                        elif down7+down8 == 2:
                            positive_train.append([x_ini+i,y_ini+j,z_ini+k])
                        else:
                            positive_train.append([x_ini+i,y_ini+j,z_ini+k])

                elif target_3d_1[x_ini+i,y_ini+j,z_ini+k] <= low:
                    positive_must.append([x_ini+i,y_ini+j,z_ini+k])
                    
    return positive_train,positive_must

low = 60
high = 90
ready = False
target_3d_1 = target_same_insert
x_ini,y_ini,z_ini = 465,334,53
i,j,k = 0,0,0
threshold=15
up_down=5
#if target_3d_1[x_ini,y_ini,z_ini]>50:
mip_threshold1 = 4
mip_threshold2 = 6.5
# else:
#     mip_threshold = 4
print(target_3d_1[x_ini,y_ini,z_ini])
if low < target_3d_1[x_ini,y_ini,z_ini] <= high:
    for n in range(int(up_down)):
        print(target_3d_1[x_ini+i,y_ini+j,z_ini+k-int((up_down-1)/2)+n])
        if target_3d_1[x_ini+i,y_ini+j,z_ini+k]+mip_threshold2<=target_3d_1[x_ini+i,y_ini+j,z_ini+k-int((up_down-1)/2)+n] and -int((up_down-1)/2)+n != 0:
            #positive_train.append([x_ini+i,y_ini+j,z_ini+k])
            print('mip')
            ready = 2
            break
        elif target_3d_1[x_ini+i,y_ini+j,z_ini+k]+mip_threshold1<=target_3d_1[x_ini+i,y_ini+j,z_ini+k-int((up_down-1)/2)+n] and -int((up_down-1)/2)+n != 0:
            print('mip')
            ready = 1
            break
    if ready == 2:
        gradient_threshold = 6
    elif ready == 1:
        gradient_threshold = 5
    else:
        gradient_threshold = 4

    #从上到下

    down1 = 0
    for m in range(4,-1,-1):
        dy1 = target_3d_1[x_ini+i,y_ini+j+m,z_ini+k]-target_3d_1[x_ini+i,y_ini+j+m+1,z_ini+k]
        if dy1 < -threshold:
            down1 = 1

        elif dy1 > threshold:
            down1 = 0


    #从下到上

    down2 = 0
    for m in range(-4,1):
        dy2 = target_3d_1[x_ini+i,y_ini+j+m,z_ini+k]-target_3d_1[x_ini+i,y_ini+j+m-1,z_ini+k]
        if dy2 < -threshold:
            down2 = 1

        elif dy2 > threshold:
            down2 = 0


    #从左到右

    down3 = 0
    for m in range(-4,1):
        dx3 = target_3d_1[x_ini+i+m,y_ini+j,z_ini+k]-target_3d_1[x_ini+i+m-1,y_ini+j,z_ini+k]
        if dx3 < -threshold:
            down3 = 1

        elif dx3 > threshold:
            down3 = 0


    #从右到左

    down4 = 0
    for m in range(4,-1,-1):
        dx4 = target_3d_1[x_ini+i,y_ini+j+m,z_ini+k]-target_3d_1[x_ini+i,y_ini+j+m+1,z_ini+k]
        if dx4 < -threshold:
            down4 = 1

        elif dx4 > threshold:
            down4 = 0

    #左斜

    down5 = 0
    for m in range(-4,1):
        dxy5 = target_3d_1[x_ini+i+m,y_ini+j-m,z_ini+k]-target_3d_1[x_ini+i+m-1,y_ini+j-m+1,z_ini+k]
        if dxy5 < -threshold:
            down5 = 1

        elif dxy5 > threshold:
            down5 = 0

    down6 = 0
    for m in range(4,-1,-1):
        dxy6 = target_3d_1[x_ini+i+m,y_ini+j-m,z_ini+k]-target_3d_1[x_ini+i+m+1,y_ini+j-m-1,z_ini+k]
        if dxy6 < -threshold:
            down6 = 1

        elif dxy6 > threshold:
            down6 = 0


    #右斜

    down7 = 0
    for m in range(-4,1):
        dxy7 = target_3d_1[x_ini+i+m,y_ini+j+m,z_ini+k]-target_3d_1[x_ini+i+m-1,y_ini+j+m-1,z_ini+k]
        if dxy7 < -threshold:
            down7 = 1

        elif dxy7 > threshold:
            down7 = 0

    down8 = 0
    for m in range(4,-1,-1):
        dxy8 = target_3d_1[x_ini+i+m,y_ini+j+m,z_ini+k]-target_3d_1[x_ini+i+m+1,y_ini+j+m+1,z_ini+k]
        if dxy8 < -threshold:
            down8 = 1

        elif dxy8 > threshold:
            down8 = 0


    down_matrix = [down1,down2,down3,down4,down5,down6,down7,down8]
    print(down_matrix)
    down_matrix.sort()
    print(down_matrix[gradient_threshold],down_matrix[gradient_threshold+1])

    if down_matrix[gradient_threshold]==0:
        pass
#     elif down_matrix[gradient_threshold+1]==0 and target_3d_1[x_ini+i+m,y_ini+j+m,z_ini+k]<=50:
#         pass
    else:
        print('get')
#             if down1+down2 == 2:
#                 positive_train.append([x_ini+i,y_ini+j,z_ini+k])
#             elif down3+down4 == 2:
#                 positive_train.append([x_ini+i,y_ini+j,z_ini+k])
#             elif down5+down6 == 2:
#                 positive_train.append([x_ini+i,y_ini+j,z_ini+k])
#             elif down7+down8 == 2:
#                 positive_train.append([x_ini+i,y_ini+j,z_ini+k])
#             else:
#                 positive_train.append([x_ini+i,y_ini+j,z_ini+k])

elif target_3d_1[x_ini+i,y_ini+j,z_ini+k] <= low:
    positive_must.append([x_ini+i,y_ini+j,z_ini+k])

In [17]:
from itertools import combinations_with_replacement
import numpy as np
from scipy import ndimage as ndi
#from torch import absolute_eigenvaluesh
from scipy import absolute
def compute_3d_hessian_matrix(nd_array, sigma=1, scale=True, whiteonblack=True):
    """
    Computes the hessian matrix for an nd_array.
    This can be used to detect vesselness as well as other features.
    In 3D the first derivative will contain three directional gradients at each index:
    [ gx,  gy,  gz ]
    The Hessian matrix at each index will then be equal to the second derivative:
    [ gxx, gxy, gxz]
    [ gyx, gyy, gyz]
    [ gzx, gzy, gzz]
    The Hessian matrix is symmetrical, so gyx == gxy, gzx == gxz, and gyz == gzy.
    :param nd_array: n-dimensional array from which to compute the hessian matrix.
    :param sigma: gaussian smoothing to perform on the array.
    :param scale: if True, the hessian elements will be scaled by sigma squared.
    :return: hessian array of shape (..., ndim, ndim)
    """
    ndim = nd_array.ndim

    # smooth the nd_array
    smoothed = ndi.gaussian_filter(nd_array, sigma=sigma, mode='nearest', truncate=3.0)

    # compute the first order gradients
    gradient_list = np.gradient(smoothed)

    # compute the hessian elements
    hessian_elements = [np.gradient(gradient_list[ax0], axis=ax1)
                        for ax0, ax1 in combinations_with_replacement(range(ndim), 2)]

    if sigma > 0 and scale:
        # scale the elements of the hessian matrix
        if whiteonblack:
            hessian_elements = [(sigma ** 2) * element for element in hessian_elements]
        else:
            hessian_elements = [-1 * (sigma ** 2) * element for element in hessian_elements]

    # create hessian matrix from hessian elements
    hessian_full=[[()] * ndim for x in range(ndim)]
    #hessian_full = [[None] * ndim] * ndim

    for index, (ax0, ax1) in enumerate(combinations_with_replacement(range(ndim), 2)):
        element = hessian_elements[index]
        hessian_full[ax0][ax1] = element
        if ax0 != ax1:
            hessian_full[ax1][ax0] = element

    hessian_rows = list()
    for row in hessian_full:
        #print(row.shape)
        hessian_rows.append(np.stack(row, axis=-1))

    hessian = np.stack(hessian_rows, axis=-2)
    return hessian


def absolute_3d_hessian_eigenvalues(nd_array, sigma=1, scale=True, whiteonblack=True):
    """
    Eigenvalues of the hessian matrix calculated from the input array sorted by absolute value.
    :param nd_array: input array from which to calculate hessian eigenvalues.
    :param sigma: gaussian smoothing parameter.
    :param scale: if True hessian values will be scaled according to sigma squared.
    :return: list of eigenvalues [eigenvalue1, eigenvalue2, ...]
    """
    return np.absolute(compute_3d_hessian_matrix(nd_array, sigma=sigma, scale=scale, whiteonblack=whiteonblack))

In [18]:
def region_adaptive(ifile,point,step=5,ratio=0.2):
    x,y,z = point
    #print(ifile[x,y,z])
    protrude = False
    region = ifile[x-step:x+step,y-step:y+step,z]
    a,b = region.shape
    #print(a,b)
    semble = []
    for i in range(a):
        for j in range(b):
            semble.append(region[i,j])
    #print(len(semble))
    semble.sort()
    #print(semble)
    #max(semble)
#     for n in range(len(semble)):
#         if semble[n]==ifile[x,y,z]:
#             print(n/len(semble))
#             break
    for n in range(len(semble)):
        #print(int(semble[n]),int(ifile[x,y,z]))
        if int(semble[n])==int(ifile[x,y,z]):
            #print(n/len(semble))
            if n<len(semble)*ratio:
                protrude = True
                break
    return protrude

In [19]:
start = time.time()
positive_semble1,positive_must1 = MIP_and_gradient(target_same_insert,left_x,right_x,left_low,left_high,left=-1)
end = time.time()
print(end-start)

73.13272714614868


In [20]:
start = time.time()
positive_semble2,positive_must2 = MIP_and_gradient(target_same_insert,left_x,right_x,right_low,right_high,left=1)
end = time.time()
print(end-start)

66.62714099884033


In [21]:
a,b,c = target_same_insert.shape
target_mask1 = np.zeros((a,b,c))

for n in range(len(positive_semble1)):
    x,y,z = positive_semble1[n]
    protrude = region_adaptive(target_same_insert,positive_semble1[n],step=5)
    if protrude == True and x<left_x:
        target_mask1[x,y,z] = 255
    
for n in range(len(positive_must1)):
    x,y,z = positive_must1[n]
    if x<left_x:
        target_mask1[x,y,z] = 255
        
for n in range(len(positive_semble2)):
    x,y,z = positive_semble2[n]
    protrude = region_adaptive(target_same_insert,positive_semble2[n],step=5,ratio=0.225)
    if protrude == True and x>right_x:
        target_mask1[x,y,z] = 255
    
for n in range(len(positive_must2)):
    x,y,z = positive_must2[n]
    if x>right_x:
        target_mask1[x,y,z] = 255

[172,249,44] in positive_semble1

In [22]:
affine = np.diag([1, 2, 3,1])
img_3d = nib.Nifti1Image(target_mask1, affine)
img_3d.shape
nib.save(img_3d, file_path1+file_number+r'\dp'+file_number+r'_mask1.nii')

In [23]:
def get_max(array):
    max_index = np.unravel_index(np.argmax(array, axis=None), array.shape)
    max_value = array[max_index]
    return max_value

In [36]:
#@jit(nopython=True)
def predict(origin,origin_pha,target1,semble,patchx,patchy,patchz,thresould_low,thresould_high,left=-1):
    #a,b,c = origin.shape
    #target = np.zeros((a,b,c),dtype=int)
    #a,b,c = target.shape
    #print(a,b,c)
    target=target1
    a,b,c = target1.shape
    #model_artifact = load_model(r'E:\article3\样例文件\Branches\S57\micro_artifact1.h5')
    #model_micro = load_model(r'E:\article3\样例文件\Branches\S57\micro_vessel.h5')
    if windows == 1:
        model_complex = load_model(r'E:\article4\样例文件\CNN\dpvein_multi_input_2_98_93.h5')
    else:
        model_complex = load_model(r'/home/zxli/ssjm/dpvein_multi_input_2_98_93.h5')

    #gc.disable()
#     semble = []
#     for x in range(int(patchx/2)+30,int(a/2),2):
#         #print(x)
#         for y in range(int(patchx/2)+30,b-int(patchx/2)+30,2):
#             for z in range(int(patchx/2),c-int(patchx/2)-1,2):
#                 semble.append([x,y,z])
#     for x in range(90,190,2):
#         for y in range(200,280,2):
#             for z in range(120,160,2):
#                 semble.append([x,y,z])
    print(len(semble))
    for n in range(len(semble)):
        if n%1000 == 0:
            print(n/1000)
        #print(semble[n])
        x,y,z = semble[n]
        #print([x,y,z])
        #result = 0
        if zb1<z<min(c-33,c-zb2) and yb1<y<yb2 and ((x<left_x) or (x>right_x)):
            if origin[2][x,y,z] >= thresould_high:
                #target[x,y,z] = 0
    #             for i,j,k in d3_plat_8_no0:
    #                 if origin[x+i,y+j,z+k]<45:
    #                     target[x,y,z] = 255
                target[x,y,z] = 255
            elif origin[2][x,y,z] <= thresould_low:# and z<=50:
                target[x,y,z] = 0

            else:
                #print(1)
                test_semble,test_pha_semble,test_big_semble,test_hessian_semble = make_region([[x,y,z]],origin,origin_pha)
                #print(test_semble[0][0].shape)
                #print(test_semble[0][0].shape)
                if len(test_semble)>0:
                    if test_semble[0][0].shape[0]==15 and test_semble[0][0].shape[1]==15 and test_semble[0][0].shape[2]==15:
                        img_complex1 = resize(test_semble[0][0], output_shape=(1,patch2,patch1,patch0,1))
                        img_complex2 = resize(test_semble[0][1], output_shape=(1,patch2,patch1,patch0,1))
                        img_complex3 = resize(test_semble[0][2], output_shape=(1,patch2,patch1,patch0,1))
                        img_complex4 = resize(test_semble[0][3], output_shape=(1,patch2,patch1,patch0,1))
                        img_complex5 = resize(test_semble[0][4], output_shape=(1,patch2,patch1,patch0,1))
                        img_complex1_pha = resize(test_pha_semble[0][0], output_shape=(1,patch2,patch1,patch0,1))
                        img_complex2_pha = resize(test_pha_semble[0][1], output_shape=(1,patch2,patch1,patch0,1))
                        img_complex3_pha = resize(test_pha_semble[0][2], output_shape=(1,patch2,patch1,patch0,1))
                        img_complex4_pha = resize(test_pha_semble[0][3], output_shape=(1,patch2,patch1,patch0,1))
                        img_complex5_pha = resize(test_pha_semble[0][4], output_shape=(1,patch2,patch1,patch0,1))
                        img_complex3b = resize(test_big_semble[0][0], output_shape=(1,patch5,patch4,patch3,1))
                        img_hessian = resize(test_hessian_semble[0], output_shape=(1,3,3,3,1))
                        prediction_complex = model_complex.predict([img_complex1,img_complex2,img_complex3,img_complex4,img_complex5,img_complex1_pha,img_complex2_pha,img_complex3_pha,img_complex4_pha,img_complex5_pha,img_complex3b,img_hessian])


                        if prediction_complex[0][1] <= 0.5:
                            target[x,y,z] = 0
#                 for i,j,k in d3_plat_8_no0:
#                     if origin[x+i,y+j,z+k]>45:
#                         target[x+i,y+j,z+k] = 0
            
    #gc.enable()
    
    #mask1,mask2,mask3,mask4 = np.vsplit(target,4)
                        
    return target

In [25]:
def m_filter_0(ifile,x,y,z,step=3):
    sum_s=[]
    for i in range(-int(step/2),int(step/2)+1):
        for j in  range(-int(step/2),int(step/2)+1):
            for k in  range(-int(step/2),int(step/2)+1):
                sum_s.append(ifile[x+i][y+j][z+k])
    sum_s.sort()
    #print(sum_s)
    return sum_s[int(step*step*step/2)+5]

In [26]:
a,b,c = target_mask1.shape
target_mask3 = np.zeros((a,b,c))

for x in range(1,a-1):
    for y in range(1,b-1):
        for z in range(1,c-1):
            if target_mask1[x,y,z] == 0:
                target_mask3[x,y,z] = m_filter_0(target_mask1,x,y,z)
            else:
                target_mask3[x,y,z] = 255

In [27]:
affine = np.diag([1, 2, 3,1])
img_3d = nib.Nifti1Image(target_mask3, affine)
img_3d.shape
nib.save(img_3d, file_path1+file_number+r'\dp'+file_number+r'_mask3.nii')

In [28]:
def m_filter_1(ifile,x,y,z,step=3):
    sum_s=[]
    for i in range(-int(step/2),int(step/2)+1):
        for j in  range(-int(step/2),int(step/2)+1):
            for k in  range(-int(step/2),int(step/2)+1):
                sum_s.append(ifile[x+i][y+j][z+k])
    sum_s.sort()
    #print(sum_s)
    return sum_s[int(step*step*step/2)+1]

In [30]:
a,b,c = target_mask1.shape
target_mask4 = np.zeros((a,b,c))

for x in range(1,a-1):
    for y in range(1,b-1):
        for z in range(1,c-1):
            if target_mask3[x,y,z] == 255:
                target_mask4[x,y,z] = m_filter_0(target_mask3,x,y,z)
            else:
                target_mask4[x,y,z] = 0

In [31]:
affine = np.diag([1, 2, 3,1])
img_3d = nib.Nifti1Image(target_mask4, affine)
img_3d.shape
nib.save(img_3d, file_path1+file_number+r'\dp'+file_number+r'_mask4.nii')

In [32]:
def make_groups(origin,g_number,n_number):
    groups = []
    now_points = []
    repeats = []
    g = 0
    number = -1
    for x,y,z in origin:
        number = number+1
        if number%1000 == 0:
            print(number)
        #if g >= 1:
            #break
        
        #length_group = len(repeats)#开始迭代时分组个数
        #print([x,y,z])
        if [x,y,z] not in repeats:#确定不在其他小组中
            if [x,y,z] not in now_points:#确定不会重复统计本小组中的点
                already_points = []

                if len(now_points) == 0:#如果是小组中第一个点，就干脆把周围一圈的点都计算进来
                    now_points.append([x,y,z])
                    for i,j,k in d3_plat_1_no0:
                        if [x+i,y+j,z+k] in origin:
                            now_points.append([x+i,y+j,z+k])
                            already_points.append([x+i,y+j,z+k])
                
                for iteration in range(n_number):
                    length = len(now_points)#开始迭代时当前小组坐标点个数
                    #print(iteration)
                    temporary_points = []

                    for i,j,k in d3_plat_1_no0:#如果不是小组中第一个点，就判断周围是不是有这个小组的点，如果有就加入进来
                        for x1,y1,z1 in already_points:
                            if [x1+i,y1+j,z1+k] in origin:
                                if [x1+i,y1+j,z1+k] not in now_points:
                                    now_points.append([x1+i,y1+j,z1+k])
                                    temporary_points.append([x1+i,y1+j,z1+k])

                    already_points = []
                    for x,y,z in temporary_points:
                        already_points.append([x,y,z])

                    if len(now_points) == length:#如果坐标点个数没有增加就结束迭代 
                        #print(length)
                        groups.append(now_points)#将不再增加的这一小组坐标点作为一个小组加入到分组中
                        for xxx,yyy,zzz in groups[g]:#将这一小组列入已经计算的小组，防止重复计算
                            repeats.append([xxx,yyy,zzz])
                        
                        now_points = []#这一小组坐标点归零，准备开始新的一个小组
                        g = g+1
                        #print(g)
                        break
            
    return groups

In [33]:
a,b,c = target_mask4.shape
#target_3d_4 = np.zeros((a,b,c))

sembles = []
for x in range(a):
    for y in range(b):
        for z in range(c):
            if target_mask4[x,y,z] == 255:
                sembles.append([x,y,z])

In [34]:
len(sembles)

501760

In [ ]:
groups = make_groups(sembles,10000,1000000)

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83000
84000
85000
86000
87000
88000
89000
90000
91000
92000
93000
94000
95000
96000
97000
98000
99000
100000
101000
102000
103000
104000
105000
106000
107000
108000
109000
110000
111000
112000
113000
114000
115000
116000
117000
118000
119000
120000
121000
122000
123000
124000
125000
126000
127000
128000
129000
130000
131000
132000
133000


In [ ]:
groups2 = groups.copy()
len(groups2)

In [40]:
a,b,c = target_3d_t1.shape
target_t1_region = np.zeros((a,b,c))
for x in range(xt1,xt2):
    for y in range(yt1,yt2):
        for z in range(zt1,zt2):
            if t1_type == 0:
                if target_3d_t1[x,y,z] < t1_threshold:
                    target_t1_region[x,y,z] = 255
            else:
                if target_3d_t1[x,y,z] > t1_threshold:
                    target_t1_region[x,y,z] = 255

In [41]:
affine = np.diag([1, 2, 3,1])
img_3d = nib.Nifti1Image(target_t1_region, affine)
img_3d.shape
nib.save(img_3d, file_path1+file_number+r'\dp'+file_number+r'_t1_region.nii')

In [42]:
a,b,c = target_t1_region.shape
csf_sembles = []
for x in range(a):
    for y in range(b):
        for z in range(c):
            if target_t1_region[x,y,z] == 255:
                csf_sembles.append([x,y,z])
print(len(csf_sembles))
csf_groups = make_groups(csf_sembles,10000,1000000)

43451
0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000


In [43]:
for n in range(len(csf_groups)):
    if len(csf_groups[n])>10000:
        print(n,len(csf_groups[n]),csf_groups[n][0])
        choose_number = n
        break

44 29971 [90, 109, 200]


In [44]:
def get_region_surface(ifile,group):
    surface_group = []
    for n in range(len(group)):
        x,y,z = group[n]
        number = 0
        for i,j,k in d3_plat_1_no0:
            #print([x+i,y+j,z+k],number)
            if ifile[x+i,y+j,z+k]==0:
                number += 1
        if number>6:
            surface_group.append([x,y,z])
    return surface_group

In [45]:
t1_surface_group = get_region_surface(target_t1_region,csf_groups[choose_number])

In [46]:
a,b,c = target_3d_t1.shape
target_t1_surface = np.zeros((a,b,c))
for n in range(len(t1_surface_group)):
    x,y,z = t1_surface_group[n]
    target_t1_surface[x,y,z] = 255

In [47]:
affine = np.diag([1, 2, 3,1])
img_3d = nib.Nifti1Image(target_t1_surface, affine)
img_3d.shape
nib.save(img_3d, file_path1+file_number+r'\dp'+file_number+r'_t1_surface.nii')

In [48]:
def expand_3d(ifile,group,min_d,max_d):
    expand_surface = []
    a,b,c = ifile.shape
    for x in range(80,a-80):
        print(x)
        for y in range(100,b-100):
            for z in range(200,c-70):
                if [x,y,z] not in group:
                    array = ifile[x-int(max_d+1):x+int(max_d+1),y-int(max_d+1):y+int(max_d+1),z-int(max_d+1):z+int(max_d+1)]
                    a1,b1,c1 = array.shape
                    array_group = []
                    near = False
                    for x1 in range(a1):
                        for y1 in range(b1):
                            for z1 in range(c1):
                                if array[x1,y1,z1] == 255 and [x-int(max_d+1)+x1,y-int(max_d+1)+y1,z-int(max_d+1)+z1] in csf_groups[choose_number]:
                                    near = True
                                    array_group.append([x-int(max_d+1)+x1,y-int(max_d+1)+y1,z-int(max_d+1)+z1])
                    if near == True:
                        min_distance = 10000
                        for n in range(len(array_group)):
                            #print(array_group[n])
                            x1,y1,z1 = array_group[n]
                            #if (x1-int(a/2))*(x-int(a/2))>0 and (y1-int(b/2))*(y-int(b/2))>0:

                            distance = get_distance_3d(x,y,z,x1,y1,z1)
                            if distance<min_distance:
                                min_distance = distance
                        if min_d<=min_distance<=max_d:
                            expand_surface.append([x,y,z])
    return expand_surface

In [49]:
def get_distance_3d(x1,y1,z1,x2,y2,z2):
    distance_3d = math.sqrt((x1-x2)**2+(y1-y2)**2+((z1-z2))**2)
    return distance_3d

In [78]:
begin_time = time.time()
expand_surface_group = expand_3d(target_t1_region,csf_groups[choose_number],7,9)
print(time.time()-begin_time)

80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
97779.75346159935


In [79]:
a,b,c = target_3d_t1.shape
target_t1_surface_expand = np.zeros((a,b,c))
for n in range(len(expand_surface_group)):
    x,y,z = expand_surface_group[n]
    target_t1_surface_expand[x,y,z] = 255

In [80]:
affine = np.diag([1, 2, 3,1])
img_3d = nib.Nifti1Image(target_t1_surface_expand, affine)
img_3d.shape
nib.save(img_3d, file_path1+file_number+r'\dp'+file_number+r'_t1_surface_expand.nii')

In [81]:
def bilinear(org_img, org_shape, dst_shape):
    dst_img = np.zeros((dst_shape[0], dst_shape[1], dst_shape[2]))
    dst_h, dst_w, dst_d = dst_shape
    org_h, org_w, org_d = org_shape
    for i in range(dst_h):
        print(i)
        for j in range(dst_w):
            for k in range(dst_d):
                src_x = i * float(org_h / dst_h)
                src_y = j * float(org_w / dst_w)
                src_z = k * float(org_d / dst_d)
                src_x_int = i * org_h // dst_h
                src_y_int = j * org_w // dst_w
                src_z_int = k * org_d // dst_d
                a = src_x - src_x_int
                b = src_y - src_y_int
                c = src_z - src_z_int

                if src_x_int+1 == org_h or src_y_int+1 == org_w or src_z_int+1 == org_d:
                    dst_img[i, j, k] = org_img[src_x_int, src_y_int, src_z_int]
                    continue
                # print(src_x_int, src_y_int)
                dst_img[i, j, k] = (1. - a) * (1. - b) * (1. - c) * org_img[src_x_int+1, src_y_int+1, src_z_int] + \
                                   (1. - a) * b * (1. - c) * org_img[src_x_int, src_y_int+1, src_z_int] + \
                                    a * (1. - b) * (1. - c) * org_img[src_x_int+1, src_y_int, src_z_int] + \
                                    a * b * (1. - c) * org_img[src_x_int, src_y_int, src_z_int] + \
                                    (1. - a) * (1. - b) * c * org_img[src_x_int+1, src_y_int+1, src_z_int] + \
                                   (1. - a) * b * c * org_img[src_x_int, src_y_int+1, src_z_int] + \
                                    a * (1. - b) * c * org_img[src_x_int+1, src_y_int, src_z_int] + \
                                    a * b * c * org_img[src_x_int, src_y_int, src_z_int]
    
    return dst_img

target_t1_surface_expand_crop = target_t1_surface_expand[coregister[0]:coregister[1],coregister[2]:coregister[3],coregister[4]:coregister[5]]
img = target_t1_surface_expand_crop
img_shape = (img.shape[0],img.shape[1],img.shape[2])
dst_shape = (target_mask4.shape[0],target_mask4.shape[1],target_mask4.shape[2])
target_t1_surface_expand_crop_insert = bilinear(img, img_shape, dst_shape)
a,b,c = target_t1_surface_expand_crop_insert.shape
#target_t1_surface_expand_crop_insert2 = np.zeros((a,b,c))
for x in range(a):
    for y in range(b):
        for z in range(c):
            if target_t1_surface_expand_crop_insert[x,y,z]>=125:
                target_t1_surface_expand_crop_insert[x,y,z]=255
            else:
                target_t1_surface_expand_crop_insert[x,y,z]=0
# for x in range(a):
#     for y in range(b):
#         for z in range(c):
#             if target_t1_surface_expand_crop_insert[x,y,z]==255:
#                 target_t1_surface_expand_crop_insert2[a-x,y,z] = 255
affine = np.diag([1, 2, 3,1]) 
img_3d = nib.Nifti1Image(target_t1_surface_expand_crop_insert, affine) 
img_3d.shape 
nib.save(img_3d,file_path+file_number+'\dp'+file_number+'_boundary.nii')

def m_filter_exp(ifile,x,y,z,step=3):
    sum_s=[]
    for i in range(-int(step/2),int(step/2)+1):
        for j in  range(-int(step/2),int(step/2)+1):
            for k in  range(-int(step/2),int(step/2)+1):
                sum_s.append(ifile[x+i][y+j][z+k])
    sum_s.sort()
    #print(sum_s)
    return sum_s[int(step*step*step/2)+1]

a,b,c = target_t1_surface_expand_crop_insert.shape
target_boundary = np.zeros((a,b,c))

for x in range(1,a-1):
    print(x)
    for y in range(1,b-1):
        for z in range(1,c-1):
            if target_t1_surface_expand_crop_insert[x,y,z] == 0:
                target_boundary[x,y,z] = m_filter_exp(target_t1_surface_expand_crop_insert,x,y,z)
            else:
                target_boundary[x,y,z] = 255

affine = np.diag([1, 2, 3,1]) 
img_3d = nib.Nifti1Image(target_boundary, affine) 
img_3d.shape 
nib.save(img_3d,file_path+file_number+'\dp'+file_number+'_boundary1.nii')

In [ ]:
for n in range(len(groups2)):
    if [183,157,158] in groups2[n]:
        print(n,len(groups2[n]))

In [ ]:
start = time.time()
patch_len = 11
step = 4
a,b,c = target_mask4.shape
target_mask5 = np.zeros((a,b,c))
semble_single = []
for n in range(len(groups2)):
#     if len(groups2[n])>1000:
#         print(len(groups2[n]))
    if len(groups2[n])>=300:# and len(groups2[n])!=1054:
        for m in range(len(groups2[n])):
            x,y,z = groups2[n][m]
            #if target_mask5[x,y,z] > 50:
            target_mask5[x,y,z] = 255
    else:
        for m in range(len(groups2[n])):
            x,y,z = groups2[n][m]
            semble_single.append([x,y,z])
#             if target_same[x,y,z] > 80:
#                 target_mask6[x,y,z] = 255
            if 10<x<a-10 and 10<y<b-10 and 10<z<c-10:
                points_brights = []
                for i in range(patch_len):
                    for j in range(patch_len):
                        for k in range(patch_len):
                            points_brights.append(target_same_insert[int(x+i-((patch_len-1)/2)),int(y+j-((patch_len-1)/2)),int(z+k-((patch_len-1)/2))])
                aver_bright = sum(points_brights)/len(points_brights)
                array = target_mask4[x-step:x+step,y-step:y+step,z-step:z+step]
                result = compute_3d_hessian_matrix(array, sigma=1.5, scale=True, whiteonblack=True)
                A = np.mat(result[5][5][5])
                U = A*A.T
                lamda,hU=np.linalg.eig(U)
                #print(lamda)
                sigma=np.sqrt(lamda)
                result2 = [sigma[0],sigma[1],sigma[2]]
                #np.std(result2)
                if target_same_insert[x,y,z]/aver_bright<1.3 and target_same_insert[x,y,z]<thresould_low+10 and len(groups2[n])>50:
                    target_mask5[x,y,z] = 255
                elif np.std(result2)>10:
                    target_mask5[x,y,z] = 255
                elif target_same_insert[x,y,z]<thresould_low:
                    target_mask5[x,y,z] = 255
                    
end = time.time()
print(end-start)

In [37]:
start = time.time()
target_mask5_1 = predict(target_tests,target_tests_pha,target_mask5,semble_single,patch3,patch4,patch5,thresould_low,thresould_high,left=-1)
end = time.time()
print(end-start)

166839
0.0


KeyboardInterrupt: 

In [ ]:
a,b,c = target_mask5_1.shape
for x in range(1,a-1):
    for y in range(1,b-1):
        for z in range(1,c-1):
            if target_mask5_1[x,y,z]==0:
                target_mask5[x,y,z] = 0

In [ ]:
affine = np.diag([1, 2, 3,1])
img_3d = nib.Nifti1Image(target_mask5, affine)
img_3d.shape
nib.save(img_3d, file_path1+file_number+r'\dp'+file_number+r'_mask5.nii')

#x,y,z = 119,256,126
#x,y,z = 104,269,126
#x,y,z = 169,291,126
x,y,z = 112,290,127
step = 5
array = target_same_insert[x-step:x+step,y-step:y+step,z-step:z+step]
result = compute_3d_hessian_matrix(array, sigma=1.5, scale=True, whiteonblack=True)
A = np.mat(result[5][5][5])
U = A*A.T
lamda,hU=np.linalg.eig(U)
#print(lamda)
sigma=np.sqrt(lamda)
result2 = [sigma[0],sigma[1],sigma[2]]
print(result2)

In [ ]:
def vessel_probability_map(ifile,mask,step=5,enlarge_number=3,A=100,B=100000,C=100):
    a,b,c = ifile.shape
    ofile = np.zeros((a,b,c))
    for x in range(step+enlarge_number,a-step-enlarge_number):
        for y in range(step+enlarge_number,b-step-enlarge_number):
            for z in range(step+enlarge_number,c-step-enlarge_number):
                if mask[x,y,z] == 255:
                    vessel_probability_sum = []
                    for enlarge in range(enlarge_number):
                        array = ifile[x-step-enlarge:x+step+enlarge,y-step-enlarge:y+step+enlarge,z-step-enlarge:z+step+enlarge]
                        result = compute_3d_hessian_matrix(array, sigma=1.5, scale=True, whiteonblack=False)
                        R = np.mat(result[step+enlarge][step+enlarge][step+enlarge])
                        U = R*R.T
                        lamda,hU=np.linalg.eig(U)
                        #print(lamda)
                        sigma=np.sqrt(lamda)
                        result2 = [lamda[0],lamda[1],lamda[2]]
                        #print(result2)
                        result2.sort()
                        lamda1 = result2[0]
                        lamda2 = result2[1]
                        lamda3 = result2[2]
                        #print(lamda1,lamda2,lamda3)
                        r_alpha = abs(lamda2/lamda1)
                        r_beta = abs(lamda3)/(abs(lamda2)*abs(lamda1))
                        gama = math.sqrt(lamda1**2+lamda2**2+lamda3**2)
                        #print('r_alpha',r_alpha,r_beta,gama)
                        #print(A,B,C)
                        #print((1-np.exp(r_alpha**2/(2*A**2))))
                        if lamda2<0 or lamda3<0:
                            vessel_probability = 0
                        else:
                            vessel_probability = (1-np.exp(-r_alpha**2/(2*A**2)))*np.exp(-r_beta**2/(2*B**2))*np.exp(-gama**2/(2*C**2))
                        #print(vessel_probability)
                        vessel_probability_sum.append(vessel_probability)
                    #print(vessel_probability_sum)
                    ofile[x,y,z] = max(vessel_probability_sum)
    return ofile

In [ ]:
target_mask6 = vessel_probability_map(target_3d_nodenoise,target_mask5,step=5,A=100,B=100000,C=100)

ifile = target_3d_nodenoise
step = 5
A,B,C = 100,100000,100
x,y,z = 205, 321, 126
vessel_probability_sum = []
enlarge_number = 3
for enlarge in range(enlarge_number):
    array = ifile[x-step-enlarge:x+step+enlarge,y-step-enlarge:y+step+enlarge,z-step-enlarge:z+step+enlarge]
    result = compute_3d_hessian_matrix(array, sigma=1.5, scale=True, whiteonblack=False)
    R = np.mat(result[step+enlarge][step+enlarge][step+enlarge])
    U = R*R.T
    lamda,hU=np.linalg.eig(U)
    #print(lamda)
    sigma=np.sqrt(lamda)
    result2 = [lamda[0],lamda[1],lamda[2]]
    #print(result2)
    result2.sort()
    lamda1 = result2[0]
    lamda2 = result2[1]
    lamda3 = result2[2]
    #print(lamda1,lamda2,lamda3)
    r_alpha = abs(lamda2/lamda1)
    r_beta = abs(lamda3)/(abs(lamda2)*abs(lamda1))
    gama = math.sqrt(lamda1**2+lamda2**2+lamda3**2)
    #print('r_alpha',r_alpha,r_beta,gama)
    #print(A,B,C)
    #print((1-np.exp(r_alpha**2/(2*A**2))))
    if lamda2<0 or lamda3<0:
        vessel_probability = 0
    else:
        vessel_probability = (1-np.exp(-r_alpha**2/(2*A**2)))*np.exp(-r_beta**2/(2*B**2))*np.exp(-gama**2/(2*C**2))
    #print(vessel_probability)
    print((1-np.exp(-r_alpha**2/(2*A**2)))*np.exp(-r_beta**2/(2*B**2))*np.exp(-gama**2/(2*C**2)))
    vessel_probability_sum.append(vessel_probability)
#ofile[x,y,z] = max(vessel_probability_sum)


target_mask6[184,202,82]

In [ ]:
affine = np.diag([1, 2, 3,1])
img_3d = nib.Nifti1Image(target_mask6, affine)
img_3d.shape
nib.save(img_3d, file_path1+file_number+r'\dp'+file_number+r'_mask6.nii')

In [ ]:
a,b,c = target_mask6.shape
target_mask6_threshold = np.zeros((a,b,c))
for x in range(a):
    for y in range(b):
        for z in range(c):
            if target_mask6[x,y,z]>=0.8:
                target_mask6_threshold[x,y,z] = 255
            else:
                target_mask6_threshold[x,y,z] = 0

In [ ]:
affine = np.diag([1, 2, 3,1])
img_3d = nib.Nifti1Image(target_mask6_threshold, affine)
img_3d.shape
nib.save(img_3d, file_path1+file_number+r'\dp'+file_number+r'_mask6_threshold.nii')

In [ ]:
start = time.time()
a,b,c = target_mask5.shape
target_mask7 = np.zeros((a,b,c))
for n in range(len(groups2)):
    vessel_part = False
    number = 0
    for m in range(len(groups2[n])):
        x,y,z = groups2[n][m]
        if target_mask6_threshold[x,y,z] == 255:
            vessel_part = True
            number = number+1
    if vessel_part==True and len(groups2[n])>25:
        for m in range(len(groups2[n])):
            x,y,z = groups2[n][m]
            if x<int(a/2):
                if x<163 and number/len(groups2[n])>0.1:
                    target_mask7[x,y,z] = 255
                elif x>=163 and (number/len(groups2[n])>0.15 or len(groups2[n])>1000):
                    if target_3d_nodenoise[x,y,z]<50:
                        target_mask7[x,y,z] = 255
            else:
                if x>340 and number/len(groups2[n])>0.1:
                    target_mask7[x,y,z] = 255
                elif x<=340 and (number/len(groups2[n])>0.15 or len(groups2[n])>1000):
                    if target_3d_nodenoise[x,y,z]<65:
                        target_mask7[x,y,z] = 255
            
    else:
        pass
                    
end = time.time()
print(end-start)

target_3d_nodenoise[330,258,133]

for n in range(len(groups2)):
    if [350,275,147] in groups2[n]:
        print(n,len(groups2[n]))
        choose = n

number = 0
for m in range(len(groups2[choose])):
    x,y,z = groups2[choose][m]
    if target_mask6_threshold[x,y,z] == 255:
        number = number+1
print(number,number/len(groups2[choose]))
    #print(target_mask6_threshold[x,y,z],[x,y,z])

In [ ]:
affine = np.diag([1, 2, 3,1])
img_3d = nib.Nifti1Image(target_mask7, affine)
img_3d.shape
nib.save(img_3d, file_path1+file_number+r'\dp'+file_number+r'_mask7.nii')

file_path = file_path1
path_p02 = file_path1+file_number+'\dp'+file_number+'_mask7.nii'#图像路径，日后可以根据需要修改
p02_img = read_data(path_p02)#使用“读取图像”函数

target_mask7 = p02_img.get_fdata()
target_mask7.shape